In [1]:
import pandas as pd
import os,re
import numpy as np

In [2]:
parsed = os.listdir('Data')

In [3]:
codes = {
    2011:{'MT':130,'CN':123,'CH':119,'LC':126},
    2012:{'CN':153,'CH':154,'LC':155,'MT':156},
    2013:{'CN':188,'CH':187,'LC':189,'MT':190},
    2015:{'CN':277,'CH':273,'LC':280,'MT':284},
    2016:{'CN':307,'CH':308,'LC':309,'MT':310},
    2017:{'CN':407,'CH':408,'LC':409,'MT':410},
    2018:{'CN':463,'CH':464,'LC':465,'MT':466},
    2019:{'CN':519,'CH':520,'LC':521,'MT':522},
    2020:{'CN':604,'CH':574,'LC':584,'MT':594},
    2021:{'CN':916,'CH':886,'LC':896,'MT':906},
    2022:{'CN':1092,'CH':1062,'LC':1072,'MT':1082}
}

In [52]:
dfs = {}


def match_gab_to_df(df,gab):
    answers=[]
    languages=[]
    used = set()
    for line in df.to_dict(orient='records'):
        if line['test']=='redação':
            answers.append('')
            languages.append('portuguese')
            continue
        possible_gabarito = gab[gab['question']==line['question_number']]
        gabarito = possible_gabarito.iloc[0]
        
        if (str(gabarito['question'])+str(gabarito['test'])) in used:
            gabarito = possible_gabarito.iloc[1]
            
        answers.append(gabarito['answer'])
        languages.append(gabarito['test'] if not pd.isna(gabarito['test']) else 'portuguese')
        used.add(str(gabarito['question'])+str(gabarito['test']))
    return answers,languages


for prova in parsed:
    df = pd.read_csv('Data/'+prova)
    print(len(df))
    year = int(re.search("\d\d\d\d",prova).group())
    df['question'] = df['question'].apply(lambda question: question.upper())
    df['year'] = year
    df['question_number'] = df['question'].apply(lambda question: int(re.search("[\d]+",question).group()) if re.search("[\d]+",question) else np.nan)
    df['test']='redação'
    if 'D1_' in prova or 'ia1' in prova or 'DIA_1' in prova:
        if year < 2017:
            df.loc[df['question_number']<=45,'test'] = 'CH'
            df.loc[df['question_number']>45,'test'] = 'CN'
        else:
            df.loc[df['question_number']<=45,'test'] = 'LC'
            df.loc[df['question_number']>45,'test'] = 'CH'
    else:
        if year < 2017:
            df.loc[df['question_number']<=135,'test'] = 'LC'
            df.loc[df['question_number']>135,'test'] = 'MT'
        else:
            df.loc[df['question_number']<=135,'test'] = 'CN'
            df.loc[df['question_number']>135,'test'] = 'MT'
    #add gabarito data
    gab = pd.read_csv('GabaritosCsv/'+prova.replace('CAD','GAB')).sort_values(by=['question','test'])
    df['answer'],df['language'] = match_gab_to_df(df,gab)
    #we want to sort dataframe in the following order: english -> spanish -> portuguese then question orders
    #this is useful for the adding of microdata
    df['sort_index'] = df['language'].apply(lambda lang: 0 if lang == 'english' else 1 if lang=='spanish' else 2)
    df = df.sort_values(by=['sort_index','question_number']).drop(columns=['sort_index'])
    
    #add microdata data
    important_columns = ['CO_PROVA','NU_PARAM_A','NU_PARAM_B','NU_PARAM_C','TP_LINGUA','CO_POSICAO','CO_HABILIDADE','TX_GABARITO']
    microdata = pd.read_csv(f'microdados/ITENS_PROVA_{year}.csv',sep=';',usecols=important_columns, encoding='latin-1')
    microdata.loc[microdata['TP_LINGUA'].isna(),'TP_LINGUA']='portuguese'
    microdata.loc[microdata['TP_LINGUA']==0,'TP_LINGUA']='english'
    microdata.loc[microdata['TP_LINGUA']==1,'TP_LINGUA']='spanish'
    
#     #divide tests
    for test in df['test'].drop_duplicates().tolist():
#         print(prova+' '+ test)
        one_subject_test = df[df['test']==test].copy().reset_index(drop=True)
        if test == 'redação':
            one_subject_test.to_csv(f'ProvasComMicrodados/ENEM_{year}_{test}.csv')
            continue
#         else:
#             CO_PROVA = codes[year][test]
#             specific_microdata = microdata[microdata['CO_PROVA']==CO_PROVA].copy()
#             one_subject_test['CO_PROVA'] = CO_PROVA

#             try:
#                 output = pd.merge(one_subject_test,specific_microdata,left_on=['question_number','language'],right_on=['CO_POSICAO','TP_LINGUA'])
#                 if output.empty:
#                     raise Exception(f'Failed matching')
#             except:
#                 one_subject_test['match'] = list(one_subject_test.index+1)
#                 output = pd.merge(one_subject_test,specific_microdata,left_on=['match'],right_on=['CO_POSICAO']).drop(columns=['match'])
#                 if not (output['TX_GABARITO']!=output['answer']).sum()==0:
#                     print(f'Gabarito and answer should be the same for test {prova} {test}')
#                     #CO_PROVA = str(CO_PROVA) + 'MAYBEBROKENDONOTUSE'
#             if output.empty:
#                 print('Output empty??')
#             if len(output)!=45 and len(output)!=50:
#                 raise Exception('Inconsistent Output')
#             output.drop(columns=['CO_PROVA_x','TP_LINGUA','CO_POSICAO'],inplace=True)
#             output.rename(columns={'CO_PROVA_y':'CO_PROVA'},inplace=True)
#             output['question_number'] = output['question_number'].astype(int)
        one_subject_test['question_number']=one_subject_test['question_number'].astype(int)
        if len(one_subject_test)!=45 and len(one_subject_test)!=50:
            print(prova,len(one_subject_test),test)
        one_subject_test.to_csv(f'ProvasComMicrodados/ENEM_{year}_{test}_CO_PROVA_{CO_PROVA}.csv',index=False,encoding='utf-8')
    dfs[prova] = df

90
96
90
96
96
90
96
60
2018_PV_CAD_impresso_D2_CD11_laranja.csv 15 MT
95
90
96
90
96
90
96
90
90
90
90
96
96
96


In [41]:
prova = 'ia2_caderno6_CAD_cinza_ledor_2013.csv'
df = pd.read_csv('data/ia2_caderno6_CAD_cinza_ledor_2013.csv ',encoding='utf-8')
year=2013
df['question'] = df['question'].apply(lambda question: question.upper())
df['year'] = year
df['question_number'] = df['question'].apply(lambda question: int(re.search("[\d]+",question).group()) if re.search("[\d]+",question) else np.nan)
df['test']='redação'
if 'D1_' in prova or 'ia1' in prova or 'DIA_1' in prova:
    if year < 2017:
        df.loc[df['question_number']<=45,'test'] = 'CH'
        df.loc[df['question_number']>45,'test'] = 'CN'
    else:
        df.loc[df['question_number']<=45,'test'] = 'LC'
        df.loc[df['question_number']>45,'test'] = 'CH'
else:
    if year < 2017:
        df.loc[df['question_number']<=135,'test'] = 'LC'
        df.loc[df['question_number']>135,'test'] = 'MT'
    else:
        df.loc[df['question_number']<=135,'test'] = 'CN'
        df.loc[df['question_number']>135,'test'] = 'MT'

In [46]:
df.loc[df['test']=='LC'].sort_values('question_number')

,question,body,A,B,C,D,E,year,question_number,test
1,QUESTÃO 91,"After prison blaze kills hundreds in Honduras,...",ocorrência de um incêndio em um presídio super...,questão da superlotação nos presídios em Hondu...,investigação da morte de um oficial das Nações...,conclusão do relatório sobre a morte de mais d...,causa da morte de doze detentos em um presídio...,2013,91.0,LC
6,QUESTÃO 91,"Descrição da imagem: Na charge, dois homens, a...",aceitação imediata da provocação.,descaracterização do convite a um desafio.,sugestão de armas não convencionais para um du...,deslocamento temporal do comentário lateral.,posicionamento relaxado dos personagens.,2013,91.0,LC
2,QUESTÃO 92,National Geographic News Christine Dell’AmoreP...,"os compostos químicos da morfina, produzidos p...",os ratos e os humanos possuem a mesma via meta...,a produção de morfina em grande quantidade min...,os seres humanos têm uma predisposição genétic...,a produção de morfina é um traço incomum entre...,2013,92.0,LC
7,QUESTÃO 92,Cabra sola Gloria Fuertes Hay quien dice que s...,influenciável pela opinião das demais.,consciente de sua diferença perante as outras.,conformada por não pertencer a nenhum grupo.,corajosa diante de situações arriscadas.,capaz de transformar mau humor em pranto.,2013,92.0,LC
3,QUESTÃO 93,Steve Jobs: A Life Remembered 1955-2011 Reader...,expor as maiores conquistas da sua empresa.,descrever suas criações na área da tecnologia.,enaltecer sua contribuição para o mundo digital.,lamentar sua ausência na criação de novas tecn...,discutir o impacto de seu trabalho para a gera...,2013,93.0,LC
8,QUESTÃO 93,"Pero un día, le fue presentado a Cortés un tri...","intérprete do conquistador, possibilitando-lhe...","escrava dos espanhóis, colocando-se a serviço ...","amante do conquistador, dando origem à miscige...","voz do seu povo, defendendo os interesses polí...","maldição dos astecas, infundindo a corrupção n...",2013,93.0,LC
4,QUESTÃO 94,"Descrição da imagem: Tirinha, composta por qua...",decide tirar a neve do quintal para convencer ...,culpa o pai por exercer influência negativa na...,comenta que suas discussões com o pai não corr...,conclui que os acontecimentos ruins não fazem ...,reclama que é vítima de valores que o levam a ...,2013,94.0,LC
9,QUESTÃO 94,"Duerme negrito Duerme, duerme, negrito, que tu...",destacar o orgulho da mulher como provedora do...,evidenciar a ausência afetiva da mãe na criaçã...,retratar a precariedade das relações de trabal...,ressaltar a inserção da mulher no mercado de t...,exaltar liricamente a voz materna na formação ...,2013,94.0,LC
5,QUESTÃO 95,Do one thing for diversity and inclusion The U...,"“Lá na minha escola, aprendi a jogar capoeira ...",“Outro dia assisti na TV uma reportagem sobre ...,“Eu me inscrevi no Programa Jovens Embaixadore...,“Curto muito bater papo na internet. Meus amig...,“Pesquisei em sites de culinária e preparei um...,2013,95.0,LC
10,QUESTÃO 95,Pensar la lengua del siglo xxI Fernando Lara A...,as especificidades culturais rompem com a unid...,as variedades do espanhol têm igual relevância...,a unidade linguística do espanhol fortalece a ...,a consolidação das diferenças da língua prejud...,a unificação da língua enriquece a competência...,2013,95.0,LC
